In [17]:
import sys, os
import json
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
import ipywidgets as widgets
import plotly
import plotly.graph_objects as graph_obj
import plotly.figure_factory as ff

sys.path.append(os.path.join(os.getcwd(),'/app/utils'))
from s3 import s3store
from jsonutil import WriteDictJson, ReadDictJson

In [18]:
output = widgets.Output()
clear_output = widgets.Button(description='Clear Output')

test_select = widgets.Select(
    options=[],
    description='Test:',
    disabled=False,
    #rows=25,
)

In [19]:
trainingset='2021-02-22-14-17-19-cocoseg'

In [20]:
creds = ReadDictJson('../creds.json')
if not creds:
    output.append_stdout('Failed to load credentials file {}\n'.format(args.credentails))

s3def = creds['s3'][0]
s3 = s3store(s3def['address'], s3def['access key'], s3def['secret key'], cert_verify=False)
validation = s3def['sets']['trainingset']

test_obj = '{}/{}/tests.json'.format(validation['prefix'], trainingset)
desc_obj = '{}/{}/description.json'.format(validation['prefix'], trainingset)
testjson = s3.GetDict(validation['bucket'], test_obj)
descjson = s3.GetDict(validation['bucket'], desc_obj)

tests = pd.DataFrame(testjson)
display(tests)

,date,model,accuracy,class_similarity,similarity,confusion,images,image time,batch size,test store,test bucket
0,"03/25/2021, 18:09:50",model/2021-02-24-10-28-35-cocoseg/model-fp16.trt,0.947675,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.670111,"[[1019378753, 9697879, 4287449, 2896630], [190...",5000,0.009090,1,ipc.larson.myds.me:31994,mllib
1,"03/25/2021, 11:17:47",model/2021-02-24-10-28-35-cocoseg/jetson-fp16.trt,0.945348,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.669143,"[[1019841478, 9798289, 4154066, 2873198], [191...",5000,0.030227,1,192.168.0.155:31994,mllib
2,"03/26/2021, 06:49:37",model/2021-02-24-10-28-35-cocoseg/jetson-fp16.trt,0.941561,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.668516,"[[1019086645, 9746681, 4263134, 2923257], [192...",5000,0.047822,1,192.168.0.155:31994,mllib
3,"03/26/2021, 06:59:26",2021-02-24-10-28-35-cocoseg,0.947169,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.666395,"[[1019804297, 9893551, 4118563, 2840786], [191...",5000,0.076143,1,ipc.larson.myds.me:31994,mllib
4,"03/26/2021, 07:22:36",2021-02-24-10-28-35-cocoseg,0.947338,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.668303,"[[1019961488, 9639005, 4178823, 2874740], [191...",5000,0.075768,1,ipc.larson.myds.me:31994,mllib
5,"03/26/2021, 07:32:26",2021-02-24-10-28-35-cocoseg,0.947462,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.667802,"[[1019856431, 9680840, 4071418, 2860566], [193...",5000,0.076644,1,ipc.larson.myds.me:31994,mllib


In [21]:
iClasses = {}
classNames = [''] * descjson['classes']['classes']
for clasdesc in descjson['classes']['objects']:
    if clasdesc['trainId'] not in iClasses.keys():
        classNames[clasdesc['trainId']] = clasdesc['category']
        if not (clasdesc['category'] == 'void'):
            iClasses[clasdesc['trainId']] = clasdesc

overview = {}
results = {}
confusion = {}
test_names = []
    
for test in testjson:
    name = '{} {}'.format(test['model'], test['date'])
    test_names.append(name)
    
    test_overview = {'images':test['images'],'accuracy':test['accuracy'],'similarity':test['similarity'], 'inference time':test['image time']}
    overview[name]=test_overview
    
    similarity = {}
    for key in iClasses:
        res = test['class_similarity'][str(key)]
        if res:
            similarity[iClasses[key]['category']] = test['class_similarity'][str(key)]['similarity']

    results[name] = {'similarity':similarity, 'confusion':test['confusion']}
    
display(pd.DataFrame(overview).T)

,images,accuracy,similarity,inference time
"model/2021-02-24-10-28-35-cocoseg/model-fp16.trt 03/25/2021, 18:09:50",5000.0,0.947675,0.670111,0.009090
"model/2021-02-24-10-28-35-cocoseg/jetson-fp16.trt 03/25/2021, 11:17:47",5000.0,0.945348,0.669143,0.030227
"model/2021-02-24-10-28-35-cocoseg/jetson-fp16.trt 03/26/2021, 06:49:37",5000.0,0.941561,0.668516,0.047822
"2021-02-24-10-28-35-cocoseg 03/26/2021, 06:59:26",5000.0,0.947169,0.666395,0.076143
"2021-02-24-10-28-35-cocoseg 03/26/2021, 07:22:36",5000.0,0.947338,0.668303,0.075768
"2021-02-24-10-28-35-cocoseg 03/26/2021, 07:32:26",5000.0,0.947462,0.667802,0.076644


In [22]:
c = np.array(results[test_names[0]]['confusion'])
norm_confusion = (c.T / c.astype(np.float).sum(axis=1)).T
norm_confusion = norm_confusion.round(decimals=3)

confusion_text = [[str(round(y, 3)) for y in x] for x in norm_confusion]

# set up figure 
confusion_plot = plotly.figure_factory.create_annotated_heatmap(norm_confusion, x=classNames, y=classNames, colorscale='plasma')

# add custom xaxis title
confusion_plot.add_annotation(dict(font=dict(color="black",size=16),
                   x=0.5,
                   y=-0.15,
                   showarrow=False,
                   text="Predicted value",
                   xref="paper",
                   yref="paper"))

# add custom yaxis title
confusion_plot.add_annotation(dict(font=dict(color="black",size=16),
                   x=-0.2,
                   y=0.5,
                   showarrow=False,
                   text="Real value",
                   textangle=-90,
                   xref="paper",
                   yref="paper"))


# adjust margins to make room for yaxis title
confusion_plot.update_layout(margin=dict(t=50, l=200))

# add colorbar
confusion_plot['data'][0]['showscale'] = True
confusion_plot.update_layout(yaxis_autorange="reversed")

confusion_display = graph_obj.FigureWidget(confusion_plot)


In [23]:
def SelectTest(change, select, display, testjson):
    global test_select
    global confusion_display
    global output
    global results
    global confusion_plot
    
    with confusion_display.batch_update():
        
        c = np.array(results[test_select.value]['confusion'])
        norm_confusion = (c.T / c.astype(np.float).sum(axis=1)).T
        confusion_text = [str(round(x, 3)) for x in norm_confusion.flatten()]
        confusion_display.data[0].z =  norm_confusion
        confusion_display.update_layout(yaxis_autorange="reversed")
        
        for i in range(len(confusion_text)):
            confusion_display.layout.annotations[i].text = confusion_text[i]
            
def ClearOutput(b):
    global test_select
    global confusion_display
    global output
    output.clear_output()

In [24]:

clear_output.on_click(lambda b: ClearOutput(b))


display_results = widgets.HBox([test_select, confusion_display])
attach = widgets.VBox([display_results, clear_output, output])

test_select.observe (lambda change:SelectTest(change, select=test_select, display=confusion_display, testjson=testjson), names="value")

if len(test_names) > 0:
    test_select.options = test_names
    test_select.value = test_names[0]

display(attach)